In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from imblearn.metrics import geometric_mean_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve,auc
from sklearn.svm import SVC
import toad

In [12]:
# Windows
train = pd.read_csv('C:/Users/93100/Downloads/titanic/train.csv')
test = pd.read_csv('C:/Users/93100/Downloads/titanic/test.csv')

# Mac
# train = pd.read_csv('/Users/Desktop/titanic/train.csv')
# test = pd.read_csv('/Users/Desktop/titanic/test.csv')

In [13]:
(train.isna().sum()/train.shape[0]).apply(lambda x:format(x, '.2%'))

PassengerId     0.00%
Survived        0.00%
Pclass          0.00%
Name            0.00%
Sex             0.00%
Age            19.87%
SibSp           0.00%
Parch           0.00%
Ticket          0.00%
Fare            0.00%
Cabin          77.10%
Embarked        0.22%
dtype: object

In [14]:
train.select_dtypes('O')

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S
...,...,...,...,...,...
886,"Montvila, Rev. Juozas",male,211536,NaN,S
887,"Graham, Miss. Margaret Edith",female,112053,B42,S
888,"Johnston, Miss. Catherine Helen ""Carrie""",female,W./C. 6607,NaN,S
889,"Behr, Mr. Karl Howell",male,111369,C148,C


In [15]:
train.select_dtypes('number')

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000
887,888,1,1,19.0,0,0,30.0000
888,889,0,3,NaN,1,2,23.4500
889,890,1,1,26.0,0,0,30.0000


In [16]:
train_process = train.set_index(['PassengerId'])
train_process = train_process.drop(['Cabin'], axis=1)

train_process['Called'] = train_process['Name'].str.findall('Miss|Mr|Ms').str[0] 

# Get the length of name
train_process['Name_length'] = train_process['Name'].apply(lambda x: len(x))

# Get the First name   
train_process['First_name'] = train_process['Name'].str.split(',').str[0]

train_process = train_process.drop(['Name'], axis=1)
train_process

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Called,Name_length,First_name
PassengerId,,,,,,,,,,,,
1,0,3,male,22.0,1,0,A/5 21171,7.2500,S,Mr,23,Braund
2,1,1,female,38.0,1,0,PC 17599,71.2833,C,Mr,51,Cumings
3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss,22,Heikkinen
4,1,1,female,35.0,1,0,113803,53.1000,S,Mr,44,Futrelle
5,0,3,male,35.0,0,0,373450,8.0500,S,Mr,24,Allen
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,male,27.0,0,0,211536,13.0000,S,NaN,21,Montvila
888,1,1,female,19.0,0,0,112053,30.0000,S,Miss,28,Graham
889,0,3,female,NaN,1,2,W./C. 6607,23.4500,S,Miss,40,Johnston


In [17]:
test_process = test.set_index(['PassengerId'])
test_process = test_process.drop(['Cabin'], axis=1)

test_process['Called'] = test_process['Name'].str.findall('Miss|Mr|Ms').str[0].to_frame() # Get the Called

# Get the length of name
test_process['Name_length'] = test_process['Name'].apply(lambda x: len(x))
test_process['First_name'] = test_process['Name'].str.split(',').str[0] # Get the First name

test_process = test_process.drop(['Name'], axis=1)
test_process

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Called,Name_length,First_name
PassengerId,,,,,,,,,,,
892,3,male,34.5,0,0,330911,7.8292,Q,Mr,16,Kelly
893,3,female,47.0,1,0,363272,7.0000,S,Mr,32,Wilkes
894,2,male,62.0,0,0,240276,9.6875,Q,Mr,25,Myles
895,3,male,27.0,0,0,315154,8.6625,S,Mr,16,Wirz
896,3,female,22.0,1,1,3101298,12.2875,S,Mr,44,Hirvonen
...,...,...,...,...,...,...,...,...,...,...,...
1305,3,male,NaN,0,0,A.5. 3236,8.0500,S,Mr,18,Spector
1306,1,female,39.0,0,0,PC 17758,108.9000,C,NaN,28,Oliva y Ocana
1307,3,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,Mr,28,Saether


In [18]:
# Fill the missing value
X_train = train_process.iloc[:, 1:] # Get the features
y_train = train_process.iloc[:, 0] # Get the target
X_test = test_process
y_test = test_process.iloc[:, 0]


tar_encode1 = TargetEncoder(cols=['Sex', 'Ticket', 'Embarked', 'Called', 'Name_length', 'First_name', 'First_name'],
                            handle_missing='value',
                            handle_unknown='value') # Target encoding

tar_encode1.fit(X_train, y_train)

X_train_encode = tar_encode1.transform(X_train)
X_test_encode = tar_encode1.transform(X_test)

X_train

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Called,Name_length,First_name
PassengerId,,,,,,,,,,,
1,3,male,22.0,1,0,A/5 21171,7.2500,S,Mr,23,Braund
2,1,female,38.0,1,0,PC 17599,71.2833,C,Mr,51,Cumings
3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss,22,Heikkinen
4,1,female,35.0,1,0,113803,53.1000,S,Mr,44,Futrelle
5,3,male,35.0,0,0,373450,8.0500,S,Mr,24,Allen
...,...,...,...,...,...,...,...,...,...,...,...
887,2,male,27.0,0,0,211536,13.0000,S,NaN,21,Montvila
888,1,female,19.0,0,0,112053,30.0000,S,Miss,28,Graham
889,3,female,NaN,1,2,W./C. 6607,23.4500,S,Miss,40,Johnston


In [20]:
# Binning
train_binning = pd.concat([X_train_encode[['Age', 'Fare']], y_train], axis=1)
test_binig = pd.concat([X_train_encode[['Age', 'Fare']], y_test], axis=1)
c_tree = toad.transform.Combiner()
c_tree.fit(train_binning, y='Survived', method='dt', min_samples=0.05, n_bins=6) # Binning is based on decision tree

train_binned = c_tree.transform(pd.concat([X_train_encode[['Age', 'Fare']], y_train], axis=1))
test_binned = c_tree.transform(pd.concat([X_test_encode[['Age', 'Fare']]], axis=1), labels=False)
                               
tar_encode2 = TargetEncoder(cols=['Age', 'Fare'], handle_missing='value', handle_unknown='value') # Target encoding
tar_encode2.fit(train_binned.iloc[:, :-1], train_binned.iloc[:, -1])

train_binned_target = tar_encode2.transform(train_binned.iloc[:, :-1])          
test_binned_target = tar_encode2.transform(test_binned)

X_train_final = pd.concat([X_train_encode.drop(['Age', 'Fare'], axis=1), train_binned_target], axis=1)
X_test_final = pd.concat([X_test_encode.drop(['Age', 'Fare'], axis=1), test_binned_target], axis=1)

In [ ]:
# Imbalance processing


In [21]:
# 
estimator_score = {}
for est in np.arange(30, 500, 10):
    score = cross_val_score(estimator=RandomForestClassifier(n_estimators=est, X=X_smo, y=y_smo, cv=10, n_jobs=-1, scoring='f1')).mean()
    estimator_score[est] = scor
    
pd.DataFrame([estimator_score], index=['n_estimators']).T.sort_values(by='n_estimators', ascending=False)

SyntaxError: invalid syntax (2549085871.py, line 4)